# Handling index fragmentation and statistics

In this recipe we will see how to receive an information about fragmentation and statistics. Then we'll use the knowledge to create automated reindexing script.

First of all, we need to prepare some data.

In [ ]:
use Demo
go

drop table if exists Fragmented
create table Fragmented
(
PrimaryKey uniqueidentifier not null primary key default(newid())
, SomeValue nvarchar(200) not null default(replicate('a', rand() * 100))
)
go

set nocount ON
insert Fragmented default VALUES
go 10000

## Index fragmentation DMO
Information about freagmentation can be received using DMO **sys.dm\_db\_index\_physical\_stats**. How to query this function is seen on following statement.

Interesting columns are:

- object\_id
- index\_id
    - 0... heap
    - 1... clustered index
    - greater than 1... nonclustered index
- alloc\_unit\_type\_desc
    - in-row data is usually interesting
- avg\_fragmentation\_in\_percent
    - this is the column compared to percent of fragmentation less than 5, between 5 and 30, and greater than 30
- page\_count
    - very small tables or indexes (like with less than 500 pages) have confusing avg\_fragmentation\_in\_percent, we do not care about them

In [ ]:
use Demo
go

select * from sys.dm_db_index_physical_stats(db_id(), null, null, null, null)
go

The result of preceding query is not readable easily, let's enhance it as follows:

In [ ]:
use Demo
go

select
	CONCAT(OBJECT_SCHEMA_NAME(stat.object_id), '.',  OBJECT_NAME(stat.object_id)) as full_table_name
	, i.name as index_name
	, stat.avg_fragmentation_in_percent
	, stat.page_count
from sys.dm_db_index_physical_stats(db_id(), null, null, null, null) as stat
	join sys.indexes as i on i.index_id = stat.index_id and i.object_id = stat.object_id
where stat.avg_fragmentation_in_percent >= 20 
	and stat.page_count >= 100
	and stat.alloc_unit_type_desc = 'IN_ROW_DATA'

Now, when we know how objects are fragmented, we can write a loop building up a SQL statements for defragmentation

## Automated defragmentation

Following script looks weird, but it is not. The _SELECT_ statement for cursor looks for all fragmented objects. The most important part is inside the cursor loop:

1. What is the index\_id? If it equals to 0, the table is HEAP, so only *ALTER TABLE ... REBUILD* is possible statement
2. If the index\_id does not equal to 0, it's index and we need to decide if the index is fragmented more than 30 percent 
    1. If it is not, *REORGANIZE* option for *ALTER INDEX* is good enough 
    2. If it is, we should use *REBUILD* option
3. When REORGANIZE is decided, *UPDATE STATISTICS* should be added to the index maintenance

In [ ]:
use Demo
go
/*pars*/
declare @page_count int = 100
	, @less_frag int = 10
	, @hours_from_bginning int = 1

/*vars*/
declare @startTme datetime2 = sysdatetime()
	, @index_id int
	, @object_name nvarchar(200)
	, @index_name nvarchar(200)
	, @current_frag dec(5, 2)
	, @sql nvarchar(600)
declare crs cursor
for
select
	stat.index_id
	, CONCAT(OBJECT_SCHEMA_NAME(stat.object_id), '.',  OBJECT_NAME(stat.object_id)) as full_table_name
	, i.name as index_name
	, stat.avg_fragmentation_in_percent
from sys.dm_db_index_physical_stats(db_id(), null, null, null, null) as stat
	join sys.indexes as i on i.index_id = stat.index_id and i.object_id = stat.object_id
where stat.avg_fragmentation_in_percent >= @less_frag
	and stat.page_count >= @page_count
	and stat.alloc_unit_type_desc = 'IN_ROW_DATA'

open crs
fetch crs into @index_id, @object_name, @index_name, @current_frag
while @@FETCH_STATUS = 0 and SYSDATETIME() < dateadd(hh, @hours_from_bginning, @startTme)
 begin
	if @index_id = 0 -- heap
		set @sql = CONCAT('ALTER TABLE ', @object_name, ' REBUILD')
	else
	 begin
		set @sql = CONCAT('ALTER INDEX ', @index_name, ' ON ', @object_name, IIF(@current_frag < 30, ' REORGANIZE', ' REBUILD'))
		if @current_frag < 30
			set @sql += ' UPDATE STATISTICS ' + @object_name
	 end
	print @sql
    -- exec(@sql)
	fetch crs into @index_id, @object_name, @index_name, @current_frag
 end
close crs
deallocate crs

When preceding script is completed, let's take a look at the fragmentation now.

In [ ]:
use Demo
go

select
	CONCAT(OBJECT_SCHEMA_NAME(stat.object_id), '.',  OBJECT_NAME(stat.object_id)) as full_table_name
	, i.name as index_name
	, stat.avg_fragmentation_in_percent
	, stat.page_count
from sys.dm_db_index_physical_stats(db_id(), null, null, null, null) as stat
	join sys.indexes as i on i.index_id = stat.index_id and i.object_id = stat.object_id
where stat.avg_fragmentation_in_percent >= 20 
	and stat.page_count >= 100
	and stat.alloc_unit_type_desc = 'IN_ROW_DATA'

## Conclusion
This recipe didn't show breaking magic, as an alternative Ola Hallengren's script is frequently used. This recipe showed more diagnostics and possible way of defragmentation.